In [1]:
# coding=utf-8
import numpy as np
import os
import pandas as pd
import csv
import sqlite3 as lite
from bs4 import BeautifulSoup
from selenium import webdriver
from shutil import copyfile
import requests
teenslang_appwords_csv = "teenslang2.csv"
test_dict_file = "test_dict.csv"
rus_letters_numbers_path = "rus_letters_numbers.npy"

all_words_links_path = 'links.npy'
vsekidki_appwords_csv = "vsekidki2"

### Functions

In [2]:
def Text_from_html_object(text):
    start = text.find("<")
    finish = text.find(">")
    text = text[:start] + text[finish + 1:]
    check_for_tag = text.find('<', start)
    while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
    return text

def Text_from_html_arr(arr):
    answer_text = ''
    for object in arr:
        text = str(object)
        check_for_tag = text.find('<')
        while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
        answer_text += text
    return answer_text

def get_page_count(soup):
    list_of_pages = soup.find_all('a', class_='page_number')
    try:
        last_page = list_of_pages[-2]
    except IndexError:
        return 1
    last_number = Text_from_html_object(str(last_page))
    return int(last_number)

def take_rus_letters_numbers(soup):
    links = []
    list_of_links = soup.find_all('a', class_='alph_menu')
    for link in list_of_links:
        pos = int(str(link).find("%D0%")) + 4
        name = str(str(link)[pos:pos+2])
        if name != "81":
            links.append(name)
    return links

def make_save(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name', 'definition', 'type', 'group', 'examples', 'origin', 'hashtags', 'synonyms'))

def add_to_save(dictionary, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dictionary:
            writer.writerow((word['name'], word['definition'], word['type'],
                             word['examples'], word['origin'], word['hashtags'], word['synonyms']))
            

    
# and then rename mycsv.out to mycsv.csv if you wish.

def get_words_from_Teenslang(from_i, to_i, file):
    make_save(file)
    for j in range(from_i, to_i + 1):
        driver.get('http://teenslang.su/content/%D0%'+ rus_letters_numbers[j]) # download site to the Selenium
        html = driver.page_source # saving site source
        soup = BeautifulSoup(html, "lxml")
        page_count = get_page_count(soup)
        print('Буква алфавита № ' + str(j + 1) + ': ')
        dict = []
        for i in range(page_count):
            if i > 0:
                driver.get('http://teenslang.su/?page=' + str(i) + '&content=%D0%'+ rus_letters_numbers[j] + '*') # load sites
                html = driver.page_source # saving site source
                soup = BeautifulSoup(html, "lxml")
            print('Парсинг %d из %d' % (i + 1, page_count))
            table = soup.find('div', class_='visible-xs visible-md visible-sm visible-lg col-lg-6')
            words = table.find_all('div', class_='issueword1')
            for word_row in words:
                word_name = word_row.find('span', class_='cap').text
                word_type = word_row.find('span', class_='sta').text
                word_group = word_row.find('span', class_='sph').text
                word_def = word_row.find('span', class_='mea').text
                word_example = word_row.find('span', class_='text').text
                word_story = word_row.find_all('div', class_='category').text
                word_hash = word_row.find('span', class_='src').text
                word_syn = word_row.find('span', class_='syn').text
                dict.append({
                    'num': i,
                    'name': brush(word_name),
                    'definition': word_def,
                    'type': " " if word_type == "NonNone" else word_type,
                    'group': " " if word_group == "NonNone" else word_group,
                    'examples': " " if word_example == "NonNone" else word_example,
                    'origin': " " if word_story == "NonNone" else word_story,
                    'hashtags': " " if word_hash == "NonNone" else word_hash,
                    'synonyms': " " if word_syn == "NonNone" else word_syn 
                })
        add_to_save(dict, file)
    fulldict = pd.read_csv(file)
    fulldict.drop_duplicates(keep='first')
    print("!!!!!!!!!!!ВСЕ!!!!!!!!!!")
            
        
def get_rus_letters_numbers():
    driver.get("http://teenslang.su/content/%D0%90")
    html = driver.page_source # saving site source
    soup = BeautifulSoup(html, "lxml")
    rus_letters_numbers = take_rus_letters_numbers(soup)
    return rus_letters_numbers

def brush(x):
    return x.capitalize().strip()

def print_dict_sorted_by_value(dictionary):
    for (key, value) in sorted(dictionary.items(), key=lambda x: x[1], reverse=True):
        print(key, ": ", value, sep='')
        
def load_webdriver():
    global driver
    options = webdriver.ChromeOptions()
    options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'
    options.add_argument('window-size=800x841')
    options.add_argument('headless')
    driver = webdriver.Chrome('/Users/lotfull/chromedriver', chrome_options=options)
#     webdriver_path = "/Users/lotfull/phantomjs/bin/phantomjs"
#     driver = webdriver.PhantomJS(webdriver_path)
        
def load_rus_letters_numbers():
    global rus_letters_numbers
    if os.path.isfile(rus_letters_numbers_path):
        rus_letters_numbers = np.load(rus_letters_numbers_path).tolist()
    else:
        rus_letters_numbers = get_rus_letters_numbers()
        np.save(rus_letters_numbers_path, rus_letters_numbers)
        

In [47]:
letter_links = ['http://teenslang.su/content/%D0%'+ letter 
                for letter in rus_letters_numbers[21:]]

In [48]:
with open ('letter_links', 'w') as links_file:
    links_file.write('\n'.join(letter_links))

In [51]:
!open .

In [50]:
!wget https://teenslang.su/index.php?adv=1&settled=yes&in_titles=yes&in_titles_first=yes&in_variants=yes

In [49]:
!aria2c -i letter_links -d letter_links_pages --continue=true


04/18 19:10:15 [NOTICE] Downloading 11 item(s)

04/18 19:10:16 [NOTICE] CUID#9 - Redirecting to https://teenslang.su/content/%D0%A7

04/18 19:10:16 [NOTICE] CUID#11 - Redirecting to https://teenslang.su/content/%D0%A9

04/18 19:10:16 [NOTICE] CUID#7 - Redirecting to https://teenslang.su/content/%D0%A5

04/18 19:10:16 [NOTICE] CUID#8 - Redirecting to https://teenslang.su/content/%D0%A6

04/18 19:10:16 [NOTICE] CUID#10 - Redirecting to https://teenslang.su/content/%D0%A8
[DL:0B][#933f85 0B/0B][#6aedeb 0B/0B][#8677de 0B/0B][#4c3573 0B/0B][#72a7fc 0B/
04/18 19:10:17 [NOTICE] File already exists. Renamed to letter_links_pages/Х.1.
[DL:450KiB][#933f85 152KiB/0B][#6aedeb 84KiB/0B][#8677de 80KiB/0B][#4c3573 80Ki
04/18 19:10:19 [NOTICE] Download complete: letter_links_pages/Х.1

04/18 19:10:19 [NOTICE] Download complete: letter_links_pages/Щ

04/18 19:10:19 [NOTICE] CUID#12 - Redirecting to https://teenslang.su/content/%D0%AA

04/18 19:10:19 [NOTICE] CUID#13 - Redirecting to https://teenslang.

In [6]:
pages_dir = !cd Teenslang-Scrapping/adv_pages/ ; pwd
pages_dir = pages_dir[0] + '/'
pages_dir

'/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/teenslang2/Teenslang-Scrapping/adv_pages/'

In [15]:
%%time

words_dict = []

def a(x):
    if x:
        return x.text
    else:
        return ''

for page in os.listdir(pages_dir)[:100]:
    with open(pages_dir + page, 'r') as word_page_file:
        word_page = word_page_file.read()
    soup = BeautifulSoup(word_page, "lxml")
    table = soup.find('div', class_='visible-xs visible-md visible-sm visible-lg col-lg-6')
    words = table.find_all('div', class_='issueword1')
    for word in words:
        words_dict.append({
            'name': brush(a(word.find('span', class_='cap'))),
            'definition': a(word.find('span', class_='mea')),
            'type': a(word.find('span', class_='sta')),
            'group': a(word.find('span', class_='sph')).strip('()'),
            'examples': a(word.find('span', class_='text')),
            'origin': a(word.find('div', class_='category')),
            'hashtags': a(word.find('span', class_='src')),
            'synonyms': a(word.find('span', class_='syn')),
            'link': 'teenslang.su/' + word.find('a', target="_blank", class_="btn btn-default btn-circle")['href']
        })

In [16]:
[x 
 for x in words_dict
 if len(x['examples'].split('\n')) > 10
]

[{'name': 'Пропуск',
  'definition': 'прием, при котором в тексте вопроса пропускается одно или несколько слов, которые обычно нужно восстановить.',
  'type': '-а, м.\xa0',
  'group': 'ИИ, термин',
  'examples': '\nПросрать\nПропуск\nПроходная машина\nПрохожий\n\nПрохилять\nпробитый\nпросраться\nПрозрачка \n\nпромот\nПрокрастинация\nПровтыкать\nпровацировать\n\nпроигрывать\nпрохиндей\nПродаван\nПрогер\n\n',
  'origin': '#Термины ',
  'hashtags': '',
  'synonyms': '',
  'link': 'teenslang.su/id/6287'},
 {'name': 'Дичь',
  'definition': 'непpивлекательная особа женского пола.',
  'type': 'ж.',
  'group': '',
  'examples': '\nдич\nДичь\nДичь\nДичь\n\nдичь\nДичь\nДичь\nДичь\n\nдичь\nДичь\nДичч\nДиспенсер\n\nдисконектит\nДивидюк\nдисомчатинг\nДичь\n\n',
  'origin': ' ',
  'hashtags': '',
  'synonyms': 'уpодина.',
  'link': 'teenslang.su/id/132'},
 {'name': 'Рак22',
  'definition': 'Конченный ублюдок, который портит игру, еблан тупой.',
  'type': 'сущ., м. р.\xa0',
  'group': 'ММОРПГ',
  'ex

In [29]:
teenslang = 'teenslang3.csv'
words = pd.DataFrame(words_dict)
print(words.shape)
words.drop_duplicates(inplace=True, keep='first')
print(words.shape)

(26401, 9)
(26231, 9)


In [31]:
words = words[['name', 'definition', 'type', 'group', 'examples', 'hashtags', 'origin', 'synonyms', 'link']]
words.to_csv(teenslang, index=False)
words.head(10)

,name,definition,type,group,examples,hashtags,origin,synonyms,link
0,Пкм,Полимерные композиционные материалы.,"существительное, Аббревиатура","технология, производство",В прошлом веке были изобретены такие ПКМ как р...,XIX век.,#Общие,,teenslang.su/id/25615
1,Плавно,"это слово говорит человек, находящимся в состо...","прил., с.р.",в компании,в разговоре:\n- тебе как?\n- мне плавно))).,когда всё вокруг медленно и классно. и ты конс...,,развозит.,teenslang.su/id/12425
2,Плаврук,"Частный случай физрука, руководит занятиями пл...","сущ., м. р.","школьники, студенты, спортсмены",,ПЛАВание + РУКоводитель.,#Общие,,teenslang.su/id/30068
3,Плаг,человек который поставляет тебе наркотики.,сущ. м.р.,употребляющие,"""Мой плаг - нарколог""\n(Gone Fludd - Lame Shit).",,,,teenslang.su/id/30144
4,Плаг-инить,В серии МегаМен Боевые сети встречается поняти...,сущ.,геймеры,После того как дошли до двери Плаг-Интесь и вы...,Plug-in - подключение к сети (англ.).,#Геймеры,Врубаться в BBS.,teenslang.su/id/14706
5,Плаги,"серьги без дырок, вставленные в тоннели.","-аг, мн.",эмо,,,#Общие,,teenslang.su/id/7621
6,Плагиат,"Чел, который говорит то, что часто говорят дру...",м.,,,,#Общие,,teenslang.su/id/4236
7,Плагиат,"Повторение, фэйк, копия оригинала.","Сущ.,М.р.","Вк,разговорная молодежная речь",-У меня такой же браслет как у тебя\n-плагиат.,,#Общие,Повторение.,teenslang.su/id/22719
8,Плагиат,Повтор чего-либо.,сущ. м.р.,все,Сплагиатил мою картинку. Ну ты и плагиат!,,#Общие #Интернет,,teenslang.su/id/23880
9,Плагиатить,Нелегально копировать без разрешения автора.,Глаг.,Ютюберы,Вася плагиат.,,#Интернет,,teenslang.su/id/23011


#### Load PhantomJS webdriver, (load from driver)/(download) numbers of Russian letters used in teenslang

In [16]:
load_webdriver()

/Users/lotfull/venvs/ml_venv/lib/python3.6/site-packages/ipykernel_launcher.py:122: DeprecationWarning: use options instead of chrome_options


In [17]:
load_rus_letters_numbers()

### test sample for letter "Й"

In [13]:
get_words_from_Teenslang(from_i=9, to_i=9, file=test_dict_file)
test_dict = pd.read_csv(test_dict_file)
test_dict.head(3)

Буква алфавита № 10: 
Парсинг 1 из 1
!!!!!!!!!!!ВСЕ!!!!!!!!!!


,name,definition,type,group,examples,origin,hashtags,synonyms
0,Йа,я.,местоимение,Йа так устала! • Когда йа была маленькой дево...,#Падонки,,,NaN
1,Йа креведко,"я знаю олбанский язык, но не совсем понимаю, о...",,,#Падонки,,"я чайник, прикидываюсь шлангом, шлангую, йа кр...",NaN
2,Йааазь,"язь, название рыбы.","-я, м.",Вот она! Вот она рыба моей мечты: здоровенный ...,#Интернет,"Из любительского видеоролика, который оказалс...","йаазь, йазь, язззь",NaN


### All words to DataFrame

In [10]:
teenslang_df = pd.read_csv(teenslang_appwords_csv)
teenslang_df.head(3)

,name,definition,type,group,examples,hashtags,origin,synonyms,id
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\n-В /p...,#Социальныесети,(происхождение второго значения) раздел был до...,_,1
1,А то,да — выражения подтверждения.,союз и частица,NaN,— Ты знаешь такого чела...? — А то!!!,NaN,NaN,да.,2
2,Абаля,вот это конфуз!.,NaN,NaN,NaN,NaN,NaN,_,3


#### Replace value of Dataframe
df.replace("#Общие ", np.nan)

In [11]:
len(teenslang_df)

16780

### Inserting ID column

In [44]:
teenslang_df["id"] = teenslang_df.index + 1
teenslang_df.head(1)

,name,definition,type,group,examples,hashtags,origin,synonyms,id
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\n-В /p...,#Социальныесети,(происхождение второго значения) раздел был до...,_,1


### Save to CSV

In [27]:
teenslang_df.to_csv(teenslang_appwords_csv, index=False)

### Find by name

In [70]:
teenslang_df[teenslang_df["name"] == "Шарить"]

,name,definition,type,group,examples,hashtags,origin,synonyms,id
16937,Шарить,"хорошо разбираться в чем-либо, понимать.",несов. перех.,NaN,Он здорово шарит в компах! • Автор не шарит в...,#Общие,NaN,рубить.,16938
16938,Шарить,"Разбираться в мемах, в моде, в том, что популя...",н.ф. глагол,(Интернет),Я шарю.,#Социальныесети,NaN,_,16939


### Drop column ID by name

In [43]:
teenslang_df = teenslang_df.drop("id", 1)
teenslang_df.head(1)

,name,definition,type,group,examples,hashtags,origin,synonyms
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\n-В /p...,#Социальныесети,(происхождение второго значения) раздел был до...,_


### Drop duplicates

In [88]:
teenslang_df.drop_duplicates(inplace=True, keep='first')
len(teenslang_df)

16751

### Repeating with Vsekidki_dict

In [7]:
vsekidki_df = pd.read_csv(vsekidki_appwords_csv)
vsekidki_df.head(1)

,name,definition,type,group,examples,origin,hashtags,synonyms,id
0,#Mannequinchallenge,"Манекен Челлендж - вирусный тренд, в котором в...",NaN,NaN,NaN,"Самый крутой челлендж, по-моему мнению, на дан...",NaN,_,1


In [ ]:
vsekidki_df = vsekidki_df.drop("id", 1)
vsekidki_df.drop_duplicates(inplace=True, keep='first')
vsekidki_df["id"] = vsekidki_df.index + 1

In [10]:
vsekidki_df.to_csv(vsekidki_appwords_csv, index=False)
len(vsekidki_df)

1307